# Analysis


Ensure that Docker is running!


## Parameters


In [2]:
MEMORY_BACKEND = "graphiti" # "memo" or "graphiti"
RESET_MEMORY_BACKENDS = True
NETWORK_PROFILE = "constrained" # "constrained" or "unconstrained"

CONVERSATION_INDEX = 0 #0
NO_OF_SESSIONS = 19 #1-19

In [3]:
NETWORK_PROFILE_UNCONSTRAINED = {
    "TOXIC_LATENCY": 0,
    "TOXIC_JITTER": 0,
    "TOXIC_BANDWIDTH": 0,
    "TOXIC_SLOW_CLOSE": 0,
    "TOXIC_TIMEOUT": 0,
    "TOXIC_SLICER": 0,
    "TOXIC_LIMIT_DATA": 0,
    "TOXIC_RESET_PEER": 0,
}

NETWORK_PROFILE_CONSTRAINED = {
    "TOXIC_LATENCY": 200, # ms
    "TOXIC_JITTER": 50, # ms
    "TOXIC_BANDWIDTH": 1000, # KB/s ≈ 1MB/s ≈ 8Mbps
    "TOXIC_SLOW_CLOSE": 0,
    "TOXIC_TIMEOUT": 0,
    "TOXIC_SLICER": 0,
    "TOXIC_LIMIT_DATA": 0,
    "TOXIC_RESET_PEER": 0,
}

if NETWORK_PROFILE == "constrained":
    PROFILE = NETWORK_PROFILE_CONSTRAINED
else:
    PROFILE = NETWORK_PROFILE_UNCONSTRAINED
PROFILE

{'TOXIC_LATENCY': 200,
 'TOXIC_JITTER': 50,
 'TOXIC_BANDWIDTH': 1000,
 'TOXIC_SLOW_CLOSE': 0,
 'TOXIC_TIMEOUT': 0,
 'TOXIC_SLICER': 0,
 'TOXIC_LIMIT_DATA': 0,
 'TOXIC_RESET_PEER': 0}

## Dependencies


In [4]:
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file in parent directory
env_path = os.path.join(os.path.dirname(os.getcwd()), '.env')
load_dotenv(env_path)

# Verify critical variables are loaded
print(f"TOXIPROXY_URL: {os.getenv('TOXIPROXY_URL', 'NOT SET')}")
print(f"LOCOMO_URL: {os.getenv('LOCOMO_URL', 'NOT SET')}")
print(f"MEMORY_BACKEND: {os.getenv('MEMORY_BACKEND', 'NOT SET')}")


TOXIPROXY_URL: http://localhost:8474
LOCOMO_URL: http://localhost:8007
MEMORY_BACKEND: graphiti


In [6]:
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


## Reset


In [1]:
!make -C ../ down

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Stopping everything on ...
cd proxy && docker-compose --env-file ../.env down || echo ""
[+] Running 0/0
 ⠋ Container ollama-proxy  Stopping                                        0.1s 
 ⠋ Container openai-proxy  Stopping                                        0.1s 
[+] Running 0/2
 ⠙ Container ollama-proxy  Stopping                                        0.2s 
 ⠙ Container openai-proxy  Stopping                                        0.2s 
[+] Running 0/2
 ⠹ Container ollama-proxy  Stopping                                        0.3s 
 ⠹ Container openai-proxy  Stopping                                        0.3s 
[+] Running 0/2
 ⠸ Container ollama-proxy  Stopping                                        0.4s 
 ⠸ Container openai-proxy  Stopping                                        0.4s 
[+] Running 1/2
 ✔ Container ollama-proxy  Removed                                         0.4s 
 ⠼ Con

In [11]:
if RESET_MEMORY_BACKENDS:
    import subprocess
    subprocess.run(["make", "-C", "../", "reset"], check=True)

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Resetting everything (keeping ollama volume)...
cd proxy && docker-compose --env-file ../.env down -v || echo ""
cd dmas && docker-compose --env-file ../.env down || echo ""
cd monitoring && docker-compose --env-file ../.env down -v || echo ""
docker volume rm dmas-long-context-memory_qdrant-data 2>/dev/null || echo ""

docker volume rm dmas-long-context-memory_neo4j-data 2>/dev/null || echo ""

docker volume rm dmas-long-context-memory_neo4j-logs 2>/dev/null || echo ""

docker volume rm prometheus-data 2>/dev/null || echo ""

docker network rm dmas-network 2>/dev/null || echo ""

Reset complete! Ollama volume preserved.
make: Leaving directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'


time="2025-11-23T17:57:22+01:00" level=warning msg="Warning: No resource found to remove for project \"proxy\"."
 Volume monitoring_prometheus-data  Removing
 Volume monitoring_prometheus-data  Removed


## Build


In [12]:
from utils import set_env_var

set_env_var('MEMORY_BACKEND', MEMORY_BACKEND)

Setting MEMORY_BACKEND=graphiti in /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env
Found existing .env file
Updated existing variable MEMORY_BACKEND
Successfully wrote to /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env


In [13]:
!make -C ../ build-monitoring

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Building monitoring services...
cd monitoring && docker-compose --env-file ../.env build --no-cache --pull
Monitoring build complete!
make: Leaving directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'


In [14]:
!make -C ../ build-proxy

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Building proxy services...
cd proxy && docker-compose --env-file ../.env build --no-cache --pull
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0/0
 ⠋ Service openai-proxy  Building                                          0.1s 
 ⠋ Service ollama-proxy  Building                                          0.1s 
[+] Building 0.2s (2/3)                                          docker:default
 => [openai-proxy internal] load build definition from Dockerfile          0.0s
 => => transferring dockerfile: 268B                                       0.0s
 => [ollama-proxy internal] load build definition from Dockerfile          0.0s
 => => transferring dockerfile: 268B                                       0.0s
 => [openai-proxy internal] load metadata for docker.io/library/python:3.  0.2s
[+] Building 0/2
 ⠙ Service openai-proxy  Building                  

In [15]:
!make -C ../ build-dmas

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Building dmas services...
cd dmas && docker-compose --env-file ../.env build --no-cache --pull
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0/0
 ⠋ Service responder  Building                                             0.1s 
 ⠋ Service memory     Building                                             0.1s 
 ⠋ Service locomo     Building                                             0.1s 
[+] Building 0.2s (3/4)                                          docker:default
 => [responder internal] load build definition from Dockerfile             0.0s
 => => transferring dockerfile: 279B                                       0.0s
 => [locomo internal] load build definition from Dockerfile                0.0s
 => => transferring dockerfile: 279B                                       0.0s
 => [memory internal] load build definition from Dockerfile          

## Run


In [8]:
!make -C ../ up

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Setting DOCKER_GID=
Starting services on ...
Docker GID:
MEMORY_BACKEND: graphiti
docker network create dmas-network 2>/dev/null || echo "Network exists"
4ef53b4d5adf99d99041984f8310e69d27e3f911bad16aeaefff7dfd8586f17f
cd proxy && docker-compose --env-file ../.env up -d --remove-orphans
[+] Running 0/0
 ⠋ Container openai-proxy  Starting                                        0.1s 
 ⠋ Container ollama-proxy  Starting                                        0.1s 
[+] Running 2/2
 ✔ Container openai-proxy  Started                                         0.1s 
 ✔ Container ollama-proxy  Started                                         0.1s 
cd dmas && docker-compose --env-file ../.env up -d --remove-orphans
[+] Running 3/0
 ✔ Network dmas-long-context-memory_memory-internal  Created               0.0s 
 ⠋ Container responder                               Starting              0.1s 
 ⠋ Container q

In [9]:
from utils import verify_memory_backend

verify_memory_backend()

/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



LOADING ENVIRONMENT VARIABLES
PROMETHEUS_URL: http://localhost:9090
TOXIPROXY_URL: http://localhost:8474
COORDINATOR_URL: http://localhost:8003


Checking MEMORY_BACKEND in container...
Container MEMORY_BACKEND: graphiti
Expected MEMORY_BACKEND: graphiti
✓ MEMORY_BACKEND correctly set to 'graphiti' in memory container



## Toxiproxy


In [10]:
import os

TOXIPROXY_URL = os.environ.get("TOXIPROXY_URL")
TOXIPROXY_URL

'http://localhost:8474'

In [11]:
from utils import apply_network_profile

apply_network_profile(PROFILE)


APPLYING NETWORK PROFILE
TOXIC_LATENCY: 200
TOXIC_JITTER: 50
TOXIC_BANDWIDTH: 1000
TOXIC_SLOW_CLOSE: 0
TOXIC_TIMEOUT: 0
TOXIC_SLICER: 0
TOXIC_LIMIT_DATA: 0
TOXIC_RESET_PEER: 0
Setting TOXIC_LATENCY=200 in /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env
Found existing .env file
Updated existing variable TOXIC_LATENCY
Successfully wrote to /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env
Setting TOXIC_JITTER=50 in /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env
Found existing .env file
Updated existing variable TOXIC_JITTER
Successfully wrote to /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env
Setting TOXIC_BANDWIDTH=1000 in /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env
Found existing .env file
Updated existing variable TOXIC_BANDWIDTH
Successfully wrote to /home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/.env
Setting TOXIC_SLOW_CLOSE=0 in

In [12]:
from utils import check_toxics

check_toxics(PROFILE, TOXIPROXY_URL)


Verifying toxic configuration...
Checking proxy 'locomo-proxy'...
  ✓ Latency: 200ms
  ✓ Jitter: 50ms
  ✓ Bandwidth: 1000.0 KB/s
Checking proxy 'memory-proxy'...
  ✓ Latency: 200ms
  ✓ Jitter: 50ms
  ✓ Bandwidth: 1000.0 KB/s
Checking proxy 'responder-proxy'...
  ✓ Latency: 200ms
  ✓ Jitter: 50ms
  ✓ Bandwidth: 1000.0 KB/s
✓ All toxics are correctly set.



## Memory


(Optional) Run `docker logs memory -f` in a terminal


### Load


In [13]:
from utils import load_memories

memories = load_memories(NO_OF_SESSIONS, CONVERSATION_INDEX, MEMORY_BACKEND)


LOADING MEMORIES: 19 session(s)
Backend: graphiti, Conversation: 0


Loading session 1/19

------------------------------------------------------------
Starting measurement for function 'load_session'
------------------------------------------------------------
No token data available
COORDINATOR_URL: http://localhost:8003
Start time: 1763918538.8285434
Loading session 1 for conversation 0
End time: 1763918848.5233216
Actual duration: 309.69s
Sleeping 2s before collecting metrics...
Current token count: 130796.0
Prometheus range query: sum(increase(docker_container_cpu_usage_total{group="edge"}[309s]))...
Result: 633749165542.3191
Prometheus range query: sum(increase(docker_container_cpu_usage_total{group="cloud"}[309s]))...
Result: 8290986610307.999
Prometheus range query: avg_over_time(docker_container_mem_usage{group="edge"}[309s])...
Result: 63757335932.203674
Prometheus range query: avg_over_time(docker_container_mem_usage{group="cloud"}[309s])...
Result: 559391033393.5828
Promet

In [14]:
from utils import export_loaded_memory_metrics

export_loaded_memory_metrics(memories, MEMORY_BACKEND, CONVERSATION_INDEX)

Exporting 19 memory metric(s)...
Constrained mode: True
Existing file backed up to results/graphiti_conv_0_sessions_constrained_backup.csv
✓ Saved new results to results/graphiti_conv_0_sessions_constrained.csv



### Ask


In [15]:
import requests
import os

LOCOMO_URL = os.getenv("LOCOMO_URL")
questions = requests.get(f"{LOCOMO_URL}/conversations/index/{CONVERSATION_INDEX}/questions").json()
print(questions)

{'conversation_index': 0, 'sample_id': 'conv-26', 'total_questions': 199, 'questions': [{'sample_id': 'conv-26', 'question_id': 'conv-26_q_0', 'question': 'When did Caroline go to the LGBTQ support group?', 'answer': '7 May 2023', 'category': 2, 'evidence': ['D1:3']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_1', 'question': 'When did Melanie paint a sunrise?', 'answer': '2022', 'category': 2, 'evidence': ['D1:12']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_2', 'question': 'What fields would Caroline be likely to pursue in her educaton?', 'answer': 'Psychology, counseling certification', 'category': 3, 'evidence': ['D1:9', 'D1:11']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_3', 'question': 'What did Caroline research?', 'answer': 'Adoption agencies', 'category': 1, 'evidence': ['D2:8']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_4', 'question': "What is Caroline's identity?", 'answer': 'Transgender woman', 'category': 1, 'evidence': ['D1:5']}, {'sa

In [20]:
from utils import run_qa

qa = run_qa(questions, MEMORY_BACKEND, CONVERSATION_INDEX, PROFILE)


RUNNING Q&A SESSION
Total questions: 199


Question 1/199: When did Caroline go to the LGBTQ support group?

------------------------------------------------------------
Starting measurement for function 'ask'
------------------------------------------------------------
Current token count: 37885.0
COORDINATOR_URL: http://localhost:8003
Start time: 1763613165.534458
End time: 1763613228.9283671
Actual duration: 63.39s
Sleeping 2s before collecting metrics...
Current token count: 38465.0
Prometheus range query: sum(increase(docker_container_cpu_usage_total{group="edge"}[63s]))...
Result: 1666973656957.1643
Prometheus range query: sum(increase(docker_container_cpu_usage_total{group="cloud"}[63s]))...
Result: 184538973187.7032
Prometheus range query: avg_over_time(docker_container_mem_usage{group="edge"}[63s])...
Result: 39405757261.20636
Prometheus range query: avg_over_time(docker_container_mem_usage{group="cloud"}[63s])...
Result: 48919987021.20636
Prometheus range query: sum(increase

KeyboardInterrupt: 

In [ ]:
from utils import export_qa

export_qa(qa, MEMORY_BACKEND, CONVERSATION_INDEX)